<a href="https://colab.research.google.com/github/201820722/SNS_HEAT/blob/main/Sns_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [3]:
cd '/content/drive/MyDrive/Colab Notebooks/Data_Mining/TP/Data'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/content/drive/MyDrive/Colab Notebooks/Data_Mining/TP/Data


In [4]:
!pip install apyori

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=c2528998f4792ddd8f11e34f32ef326eae61f7aafe8c68e3eb43be6542f5be29
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import os
import json
from apyori import apriori
import numpy as np

# 데이터 읽기 및 통합
def read_and_combine_json_files(file_paths):
    extracted_data = []

    for json_file in file_paths:
        with open(json_file, 'r') as file:
            data = json.load(file)

        extracted_item1 = []
        extracted_item2 = []
        extracted_item1.append(data['header']['dialogueInfo']['single_topic'])
        extracted_item1.append(data['header']['participantsInfo'][0]['gender'])
        extracted_item1.append(data['header']['participantsInfo'][0]['age'])
        extracted_item2.append(data['header']['dialogueInfo']['single_topic'])
        extracted_item2.append(data['header']['participantsInfo'][1]['gender'])
        extracted_item2.append(data['header']['participantsInfo'][1]['age'])
        extracted_data.append(extracted_item1)
        extracted_data.append(extracted_item2)

    df = pd.DataFrame(extracted_data, columns=['topic', 'sex', 'age'])
    # df = df[df['age'] == '20대'] ## 20대만 분석(다른 나이대 추가 가능)
    return df

# Association Rule 생성
def generate_association_rules(data):
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)
    # rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)


    return frequent_itemsets

# 기계학습 모델 구성 (TF-IDF)
def build_machine_learning_model(df):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['topic'])

    y = df['age']  # 대화 주제를 나이대로 설정

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = MultinomialNB()
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions))

    return model

# 입력에 대한 대화 트렌드 분석
def analyze_conversation_trends(input_gender, input_age, rules):
    # 입력된 성별과 나이에 따른 대화 주제 트렌드 예측
    trend_prediction = rules[(rules['antecedents'].apply(lambda x: input_gender in x)) &
                             (rules['antecedents'].apply(lambda x: input_age in x))]

    return trend_prediction['consequents']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
# 여러 개의 JSON 파일 경로 설정
json_path = '/content/drive/MyDrive/Colab Notebooks/Data_Mining/TP/Data'
json_file_paths = [pos_json for pos_json in os.listdir(json_path) if pos_json.endswith('.json')]

# 주어진 데이터 읽기
df = read_and_combine_json_files(json_file_paths)
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic,sex,age
0,개인 및 관계,여성,50대 이상
1,개인 및 관계,여성,50대 이상
2,행사,남성,30대
3,행사,여성,20대
4,개인 및 관계,여성,30대
...,...,...,...
5995,미용과 건강,여성,20대
5996,주거와 생활,여성,30대
5997,주거와 생활,여성,30대
5998,개인 및 관계,여성,20대


In [60]:
df['topic'] = df['topic'].apply(lambda x: str(x))
df['sex'] = df['sex'].apply(lambda x: str(x))
df['age'] = df['age'].apply(lambda x: str(x))
data = df
data = data.to_numpy()
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([['개인 및 관계', '여성', '50대 이상'],
       ['개인 및 관계', '여성', '50대 이상'],
       ['행사', '남성', '30대'],
       ...,
       ['주거와 생활', '여성', '30대'],
       ['개인 및 관계', '여성', '20대'],
       ['개인 및 관계', '남성', '20대']], dtype=object)

In [62]:
# res = generate_association_rules(data)
# res = pd.DataFrame(res)
# res

res = apriori(data, min_support=0.1, use_colnames=True)
res = pd.DataFrame(res, columns=['itemsets', 'support', 'ordered_statistics'])
res

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,itemsets,support,ordered_statistics
0,(20대),0.514500,"[((), (20대), 0.5145, 1.0)]"
1,(30대),0.381667,"[((), (30대), 0.38166666666666665, 1.0)]"
2,(개인 및 관계),0.275000,"[((), (개인 및 관계), 0.275, 1.0)]"
3,(남성),0.211667,"[((), (남성), 0.21166666666666667, 1.0)]"
4,(여성),0.788333,"[((), (여성), 0.7883333333333333, 1.0)]"
5,(주거와 생활),0.422333,"[((), (주거와 생활), 0.42233333333333334, 1.0)]"
6,(행사),0.128667,"[((), (행사), 0.12866666666666668, 1.0)]"
7,"(20대, 개인 및 관계)",0.136167,"[((), (20대, 개인 및 관계), 0.13616666666666666, 1.0..."
8,"(20대, 남성)",0.104167,"[((), (20대, 남성), 0.10416666666666667, 1.0), ((..."
9,"(여성, 20대)",0.410333,"[((), (여성, 20대), 0.4103333333333333, 1.0), ((2..."


In [67]:
rules = association_rules(res, metric="confidence", min_threshold=0.3)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(개인 및 관계),(20대),0.275000,0.514500,0.136167,0.495152,0.962394,-0.005321,0.961675,-0.051141
1,(남성),(20대),0.211667,0.514500,0.104167,0.492126,0.956513,-0.004736,0.955946,-0.054526
2,(여성),(20대),0.788333,0.514500,0.410333,0.520507,1.011676,0.004736,1.012529,0.054526
3,(20대),(여성),0.514500,0.788333,0.410333,0.797538,1.011676,0.004736,1.045464,0.023772
4,(20대),(주거와 생활),0.514500,0.422333,0.234333,0.455458,1.078433,0.017043,1.060831,0.149802
5,(주거와 생활),(20대),0.422333,0.514500,0.234333,0.554854,1.078433,0.017043,1.090653,0.125901
6,(개인 및 관계),(30대),0.275000,0.381667,0.113500,0.412727,1.081382,0.008542,1.052890,0.103803
7,(여성),(30대),0.788333,0.381667,0.299333,0.379704,0.994858,-0.001547,0.996836,-0.023838
8,(30대),(여성),0.381667,0.788333,0.299333,0.784279,0.994858,-0.001547,0.981208,-0.008290
9,(주거와 생활),(30대),0.422333,0.381667,0.150500,0.356354,0.933678,-0.010691,0.960672,-0.109501


In [ ]:
# 기계학습 모델 생성 및 평가
ml_model = build_machine_learning_model(rules)

In [ ]:
# 입력에 대한 대화 트렌드 분석 예시
input_gender = '남성'  # 예시 성별
input_age = '20대'    # 예시 나이
trend_prediction = analyze_conversation_trends(input_gender, input_age, rules)
print(f"Trend Prediction for {input_gender}, {input_age}:\n", trend_prediction)